In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/data.csv")
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
df.shape

(614, 13)

In [4]:
# check for duplicates in Loan_ID
df[df.duplicated(subset=['Loan_ID'], keep=False)]
print(df.shape)

#no duplicates, so drop loan ID
df = df.drop(columns="Loan_ID")

(614, 13)


Deal with NaNs

In [5]:
print(df.Education.nunique())
print(df.Gender.nunique())
print(df.Married.nunique())
print(df.Self_Employed.nunique())
print(df.Credit_History.nunique())
print(df.Property_Area.nunique())
print(df.Dependents.nunique())

2
2
2
2
2
3
4


In [6]:
df.isna().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [7]:
# 22 of 614 in LoanAmount. Drop em (too important to impute)
df = df.dropna(subset=['LoanAmount'])

In [8]:
# 14 NaNs in Loan_Amount_Term. Fill with median
median_term = df.Loan_Amount_Term.median()
print (median_term)
df.Loan_Amount_Term = df['Loan_Amount_Term'].fillna(median_term)

360.0


In [9]:
# 2 missing values for married. Fill with mode
# # mode returns a series. To assign no.1 to a variable, use index[0]
mode_married = df.Married.mode()[0]
print(mode_married)

# Fill null values in Married with the mode
df.Married = df.Married.fillna(mode_married)

Yes


In [10]:
# 13 missing values for gender. Fill with mode
# mode returns a series. To assign no.1 to a variable, use index[0]
mode_gender = df.Gender.mode()[0] 
print(mode_gender)

df.Gender = df.Gender.fillna(mode_gender)

Male


In [11]:
# 13 missing values for dependents. Fill with 0
df.Dependents = df.Dependents.fillna('0')

In [12]:
# 31 missing values for self-employed. Fill with mode
mode_employed = df.Self_Employed.mode()[0] 
print(mode_employed)
df.Self_Employed = df.Self_Employed.fillna(mode_employed)

No


In [13]:
# 50 NaNs in CreditHistory. Not sure whether to drop them or make them 0
df.Credit_History.value_counts()

1.0    458
0.0     85
Name: Credit_History, dtype: int64

In [14]:
# If 0 contained more values, I'd make them 0, but I will drop them
df = df.dropna(subset=['Credit_History'])

In [15]:
df.isna().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

Deal with Datatypes

In [16]:
df.dtypes

Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [17]:
df.Education.value_counts()

Graduate        429
Not Graduate    114
Name: Education, dtype: int64

In [18]:
df.Married.value_counts()

Yes    350
No     193
Name: Married, dtype: int64

In [19]:
# the following columns only have 2 possible values:
    # Education, Gender, Married, Self_Employed, Credit_History
# label encoding, instead of one-hot encoding, will ensure that a ton of extra columns aren't created

# import label encoding
from sklearn.preprocessing import LabelEncoder

# Create an instance of the LabelEncoder
le = LabelEncoder()

# default for label encoder is most common value is assigned to 0, less common to 1
# unless, I guess, the values are already 0 and 1 as in credit history
# or yes and no, as in self-employed

# I want to reverse this for education, so grduate = 1 and not-graduate = 0
# Fit and transform the Education column, reversing the encoding
df['Education'] = le.fit_transform(df['Education'])
df['Education'] = 1 - df['Education']

# select other columns to encode using default
cols_to_encode = ['Education', 'Married', 'Gender','Self_Employed', 'Credit_History', 'Loan_Status']

# apply LabelEncoder to selected columns
df[cols_to_encode] = df[cols_to_encode].apply(LabelEncoder().fit_transform)


In [20]:
# One-hot encode the 'Property_Area' and 'Dependents' columns
df_encoded = pd.get_dummies(df[['Property_Area', 'Dependents']])

# Concatenate the one-hot encoded columns with the original dataframe
df = pd.concat([df, df_encoded], axis=1)

# Drop the original 'Property_Area' and 'Dependents' columns
df.drop(columns=['Property_Area', 'Dependents'], inplace=True)

In [21]:
df.head()

,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Dependents_0,Dependents_1,Dependents_2,Dependents_3+
1,1,1,1,0,4583,1508.0,128.0,360.0,1,0,1,0,0,0,1,0,0
2,1,1,1,1,3000,0.0,66.0,360.0,1,1,0,0,1,1,0,0,0
3,1,1,0,0,2583,2358.0,120.0,360.0,1,1,0,0,1,1,0,0,0
4,1,0,1,0,6000,0.0,141.0,360.0,1,1,0,0,1,1,0,0,0
5,1,1,1,1,5417,4196.0,267.0,360.0,1,1,0,0,1,0,0,1,0


In [22]:
df.dtypes

Gender                       int32
Married                      int32
Education                    int64
Self_Employed                int32
ApplicantIncome              int64
CoapplicantIncome          float64
LoanAmount                 float64
Loan_Amount_Term           float64
Credit_History               int64
Loan_Status                  int32
Property_Area_Rural          uint8
Property_Area_Semiurban      uint8
Property_Area_Urban          uint8
Dependents_0                 uint8
Dependents_1                 uint8
Dependents_2                 uint8
Dependents_3+                uint8
dtype: object

Transforms

In [23]:
# export to csv
df.to_csv ("data/df_numeric.csv")
